In [1]:
import os
import requests
from os import environ as env
import dotenv
from dotenv import load_dotenv
load_dotenv()
req_headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
    }
token_server = 'https://support.econjobmarket.org/oauth2/token'
payload = {
    'client_id' : 'support_token_server',
    'client_secret' : '4support_server2work',
    'grant_type' : 'password',
    'username' : os.getenv("username"),
    'password' : os.getenv("password")
}
data = 'client_id=support_token_server&client_secret=4support_token2work&grant_type=password&username='+os.getenv("username")+'&password='+os.getenv("password")
#response = requests.post('https://support.econjobmarket.org/oauth2/token', headers=req_headers, data=payload)
#ref_token=response.json()['refresh_token']
#print(response.text)



TypeError: can only concatenate str (not "NoneType") to str

In [ ]:
print(response.json()["access_token"])

In [ ]:
dotenv_file = dotenv.find_dotenv()
dotenv.load_dotenv(dotenv_file)

print(os.environ["access_token"])  # outputs "value"
os.environ["access_token"] = response.json()["access_token"]
print(os.environ['access_token'])  # outputs 'newvalue'
os.environ["refresh_token"] = response.json()["refresh_token"]

# Write changes to .env file.
dotenv.set_key(dotenv_file, "access_token", os.environ["access_token"])
dotenv.set_key(dotenv_file, "refresh_token", os.environ["refresh_token"])
